# 13wk-2: (강화학습) – Bandit 환경 설계 및 풀이

최규빈  
2025-06-02

<a href="https://colab.research.google.com/github/guebin/DL2025/blob/main/posts/13wk-2.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" style="text-align: left"></a>

# 1. 강의영상

In [8]:
# {{<video https://youtu.be/playlist?list=PLQqh36zP38-zEjn2m8H8hMCHsQK8udE27&si=Sy-lnw4Kq56SRggu >}}

# 2. Imports

In [1]:
import gymnasium as gym
#---#
import numpy as np
import collections
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import IPython

# 3. Bandit 환경 설계 및 풀이

## A. 대충 개념만 실습

In [2]:
action_space = [0,1] 
actions_deque = collections.deque(maxlen=200)
rewards_deque = collections.deque(maxlen=200)
#---#
for _ in range(10):
    action = np.random.choice(action_space)
    if action ==0: 
        reward = 1
    else:
        reward = 10
    actions_deque.append(action)
    rewards_deque.append(reward)

In [3]:
actions_deque

In [4]:
rewards_deque

In [5]:
actions_numpy = np.array(actions_deque)
rewards_numpy = np.array(rewards_deque)

In [6]:
q0 = rewards_numpy[actions_numpy==0].mean()
q1 = rewards_numpy[actions_numpy==1].mean()
q_table = np.array([q0,q1])
q_table

In [7]:
#---#
for _ in range(5):
    #action = np.random.choice(action_space)
    action = q_table.argmax()
    if action ==0: 
        reward = 1
    else:
        reward = 10
    actions_deque.append(action)
    rewards_deque.append(reward)
    actions_numpy = np.array(actions_deque)
    rewards_numpy = np.array(rewards_deque)
    q0 = rewards_numpy[actions_numpy==0].mean()
    q1 = rewards_numpy[actions_numpy==1].mean()
    q_table = np.array([q0,q1])
    q_table

In [8]:
if rewards_numpy[-5:].mean() > 9:
    print("GameClear")

GameClear

## B. 클래스를 이용한 설계 및 풀이

In [9]:
class Batdit():
    def __init__(self):
        self.reward = None 
    def step(self,action):
        if action == 0:
            self.reward = 1
        elif action == 1:
            self.reward = 10
        return self.reward

In [10]:
class Agent():
    def __init__(self):
        self.n_experiences = 0 
        self.action_space = [0,1]
        self.action = None 
        self.actions_deque = collections.deque(maxlen=500)
        self.actions_numpy = np.array(self.actions_deque)
        self.reward = None 
        self.rewards_deque = collections.deque(maxlen=500)
        self.rewards_numpy = np.array(self.rewards_deque)
        self.q_table = None
    def act(self):
        if self.n_experiences < 20:
            self.action = np.random.choice(self.action_space)
        else: 
            self.action = self.q_table.argmax()
        print(f"버튼{self.action}누름")
    def save_experience(self):
        self.n_experiences = self.n_experiences + 1
        self.actions_deque.append(self.action)
        self.rewards_deque.append(self.reward)
        self.actions_numpy = np.array(self.actions_deque)
        self.rewards_numpy = np.array(self.rewards_deque)
    def learn(self):
        if self.n_experiences < 20:
            pass
        else: 
            q0 = self.rewards_numpy[self.actions_numpy == 0].mean()
            q1 = self.rewards_numpy[self.actions_numpy == 1].mean()
            self.q_table = np.array([q0,q1])

In [11]:
env = Batdit()
agent = Agent()

In [12]:
agent.act()

버튼1누름

In [13]:
for _ in range(100):
    #1. 행동
    agent.act()
    #2. 보상
    agent.reward = env.step(agent.action)
    #3. 저장 & 학습 
    agent.save_experience()
    agent.learn()
    #---#
    if (agent.n_experiences > 20) and (agent.rewards_numpy[-20:].mean() >9):
        print("게임클리어")
        break

버튼1누름
버튼1누름
버튼0누름
버튼0누름
버튼1누름
버튼0누름
버튼1누름
버튼1누름
버튼0누름
버튼0누름
버튼0누름
버튼1누름
버튼0누름
버튼0누름
버튼1누름
버튼1누름
버튼0누름
버튼1누름
버튼1누름
버튼1누름
버튼1누름
버튼1누름
버튼1누름
버튼1누름
버튼1누름
버튼1누름
버튼1누름
버튼1누름
버튼1누름
버튼1누름
버튼1누름
버튼1누름
버튼1누름
게임클리어

# 3. 예비학습: `gym.spaces`

ref: <https://gymnasium.farama.org/>

`-` 예시1

In [73]:
action_space = gym.spaces.Discrete(4) 
action_space 

In [74]:
[action_space.sample() for _ in range(5)]

In [75]:
0 in action_space

In [76]:
4 in action_space

`-` 예시2

In [77]:
state_space = gym.spaces.MultiDiscrete([4,4])
state_space

In [78]:
[state_space.sample() for _ in range(5)]

In [79]:
np.array([0,1]) in state_space

In [80]:
np.array([3,3]) in state_space

In [81]:
np.array([3,4]) in state_space

# 4. 4x4 Grid World 게임 설명

## A. 게임설명

`-` 문제설명: 4x4 그리드월드에서 상하좌우로 움직이는 에이전트가 목표점에
도달하도록 학습하는 방법

`-` GridWorld에서 사용되는 주요변수

1.  **`State`**: 각 격자 셀이 하나의 상태이며, 에이전트는 이러한 상태 중
    하나에 있을 수 있음.
2.  **`Action`**: 에이전트는 현재상태에서 다음상태로 이동하기 위해
    상,하,좌,우 중 하나의 행동을 취할 수 있음.
3.  **`Reward`**: 에이전트가 현재상태에서 특정 action을 하면 얻어지는
    보상.
4.  **`Terminated`**: 하나의 에피소드가 종료되었음을 나타내는 상태.

## B. 시각화

In [23]:
def show(states):
    fig = plt.Figure()
    ax = fig.subplots()
    ax.matshow(np.zeros([4,4]), cmap='bwr',alpha=0.0)
    sc = ax.scatter(0, 0, color='red', s=500)  
    ax.text(0, 0, 'start', ha='center', va='center')
    ax.text(3, 3, 'end', ha='center', va='center')
    # Adding grid lines to the plot
    ax.set_xticks(np.arange(-.5, 4, 1), minor=True)
    ax.set_yticks(np.arange(-.5, 4, 1), minor=True)
    ax.grid(which='minor', color='black', linestyle='-', linewidth=2)
    state_space = gym.spaces.MultiDiscrete([4,4])
    def update(t):
        if states[t] in state_space:
            s1,s2 = states[t]
            states[t] = [s2,s1]
            sc.set_offsets(states[t])
        else:
            s1,s2 = states[t]
            s1 = s1 + 0.5 if s1 < 0 else (s1 - 0.5 if s1 > 3 else s1)
            s2 = s2 + 0.5 if s2 < 0 else (s2 - 0.5 if s2 > 3 else s2)
            states[t] = [s2,s1]       
            sc.set_offsets(states[t])
    ani = FuncAnimation(fig,update,frames=len(states))
    display(IPython.display.HTML(ani.to_jshtml()))

In [24]:
show([[0,0],[1,0],[2,0],[3,0],[4,0]])

# 5. 4x4 Grid World 환경 구현

In [25]:
class GridWorld:
    def __init__(self):
        self.state_space = gym.spaces.MultiDiscrete([4,4])
        self.action_space = gym.spaces.Discrete(4) 
        self._action_to_direction = {
            0 : np.array([1, 0]), # row+, down
            1 : np.array([0, 1]), # col+, right
            2 : np.array([-1 ,0]), # row-, up
            3 : np.array([0, -1]) # col-, left
        }
        self.reset()
        self.state = None 
        self.reward = None 
        self.termiated = None
    def step(self,action):
        direction = self._action_to_direction[action]
        self.state = self.state + direction
        if np.array_equal(self.state,np.array([3,3])): 
            self.reward = 100 
            self.terminated = True
        elif self.state not in self.state_space:
            self.reward = -10
            self.terminated = True
        else:
            self.reward = -1 
        return self.state, self.reward, self.terminated
    def reset(self):
        self.state = np.array([0,0])
        self.terminated = False   
        return self.state 

In [26]:
env = GridWorld()
state = env.reset()
states = [] 
states.append(state)
for t in range(50):
    action = env.action_space.sample() 
    state,reward,terminated = env.step(action)
    states.append(state)
    if terminated: break 

In [27]:
show(states)

-   처음에 바로 죽는 경우가 많아 몇번 시도하고 위의 애니메이션을 얻음

# 6. `AgentRandom` 구현하여 환경과 상호작용

## A. 에이전트 클래스 설계

`-` 우리가 구현하고 싶은 기능

-   `.act()`: 액션을 결정 –\> 여기서는 그냥 랜덤액션
-   `.save_experience()`: 데이터를 저장 –\> 여기에 일단 초점을 맞추자
-   `.learn()`: 데이터로에서 학습 –\> 패스

In [82]:
class AgentRandom: 
    def __init__(self,env):
        #--# define spaces 
        self.action_space = env.action_space
        self.state_space = env.state_space
        #--# replay buffer 
        self.action = None 
        self.actions = collections.deque(maxlen=500)
        self.current_state =  None 
        self.current_states = collections.deque(maxlen=500)
        self.reward = None 
        self.rewards = collections.deque(maxlen=500)
        self.next_state =  None 
        self.next_states = collections.deque(maxlen=500)
        self.terminated = None 
        self.terminations = collections.deque(maxlen=500)
        #--# other information
        self.n_episodes = 0         
        self.n_experiences = 0
        self.score = 0        
        self.scores = collections.deque(maxlen=500)        
    def act(self):
        self.action = self.action_space.sample()
    def learn(self):
        pass 
    def save_experience(self):
        self.current_states.append(self.current_state)        
        self.actions.append(self.action)
        self.rewards.append(self.reward)  
        self.next_states.append(self.next_state)
        self.terminations.append(self.terminated)
        #--#
        self.n_experiences = self.n_experiences + 1 
        self.score = self.score + self.reward

## B. 환경과 상호작용

In [72]:
env = GridWorld()
agent = AgentRandom(env)
#--#
for _ in range(50):
    agent.current_state = env.reset()
    agent.score = 0 
    for t in range(100):
        # step1: 행동
        agent.act()
        # step2: 보상
        agent.next_state, agent.reward, agent.terminated = env.step(agent.action)
        # step3: 저장 & 학습
        agent.save_experience()
        agent.learn()
        # step4: 
        agent.current_state = agent.next_state
        if agent.terminated: break
    agent.scores.append(agent.score) 
    agent.n_episodes = agent.n_episodes + 1 
    #---#
    print(
        f"에피소드: {agent.n_episodes} \t"
        f"점수(에피소드): {agent.scores[-1]} \t" 
        f"경험수: {agent.n_experiences}"
    )

에피소드: 1     점수(에피소드): -14   경험수: 5
에피소드: 2     점수(에피소드): -10   경험수: 6
에피소드: 3     점수(에피소드): -11   경험수: 8
에피소드: 4     점수(에피소드): -13   경험수: 12
에피소드: 5     점수(에피소드): -10   경험수: 13
에피소드: 6     점수(에피소드): -10   경험수: 14
에피소드: 7     점수(에피소드): -13   경험수: 18
에피소드: 8     점수(에피소드): -12   경험수: 21
에피소드: 9     점수(에피소드): -16   경험수: 28
에피소드: 10    점수(에피소드): -10   경험수: 29
에피소드: 11    점수(에피소드): -15   경험수: 35
에피소드: 12    점수(에피소드): -10   경험수: 36
에피소드: 13    점수(에피소드): -10   경험수: 37
에피소드: 14    점수(에피소드): -11   경험수: 39
에피소드: 15    점수(에피소드): -12   경험수: 42
에피소드: 16    점수(에피소드): -12   경험수: 45
에피소드: 17    점수(에피소드): -10   경험수: 46
에피소드: 18    점수(에피소드): -10   경험수: 47
에피소드: 19    점수(에피소드): -11   경험수: 49
에피소드: 20    점수(에피소드): -10   경험수: 50
에피소드: 21    점수(에피소드): -12   경험수: 53
에피소드: 22    점수(에피소드): -10   경험수: 54
에피소드: 23    점수(에피소드): -10   경험수: 55
에피소드: 24    점수(에피소드): -12   경험수: 58
에피소드: 25    점수(에피소드): -12   경험수: 61
에피소드: 26    점수(에피소드): -10   경험수: 62
에피소드: 27    점수(에피소드): -10   경험수: 63
에피소드: 28    점수(에피소드): -11   경험수

## C. 상호작용결과 시각화

In [47]:
[np.array([0,0])] + list(agent.next_states)[134:140]

In [46]:
show([np.array([0,0])] + list(agent.next_states)[134:140])